In [1]:
aList = [1,2,3,1,1,1,2,3,4,2,2,2,2,2,5,2,2,1]
rdd = sc.parallelize(aList)
rdd.take(5)

Out[ 1 ]: [1, 2, 3, 1, 1]

In [2]:
rdd.sortBy(lambda x: -x).take(5)

Out[ 10 ]: [5, 4, 3, 3, 2]

In [3]:
rdd.filter(lambda x: x % 2 != 0).count()

Out[ 2 ]: 8

In [4]:
#rdd.takeOrdered(5, key=lambda x: -x)  OR
rdd.top(5)

Out[ 7 ]: [5, 4, 3, 3, 2]

In [5]:
rdd.stats()

Out[ 8 ]: (count: 18, mean: 2.111111111111111, stdev: 1.04822012578, max: 5.0, min: 1.0)

In [6]:
rdd1 = sc.textFile("/FileStore/tables/authors.csv")
def parse(line):
  fields = line.split(",")
  return (fields[2],fields[0])
p_rdd1 = rdd1.filter(lambda x:'Country' not in x).map(parse)
p_rdd1.take(3)

Out[ 11 ]: [('1', 'P.G. Wodehouse'), ('2', 'Arthur Conan Doyle'), ('3', 'P.G. Wodehouse')]

In [7]:
rdd2 = sc.textFile("/FileStore/tables/books.csv")
p_rdd2 = rdd2.filter(lambda line: "Title" not in line).map(lambda line: (line.split(",")[0], line.split(",")[1]))
p_rdd2.take(3)

Out[ 13 ]: 
[('1', 'The Return of Jeeves'),
 ('2', 'The Sign of Four'),
 ('3', 'Pigs Have Wings')]

In [8]:
joined_rdd = p_rdd1.join(p_rdd2)
joined_rdd.map(lambda x: x[1]).groupByKey().mapValues(list).sortByKey().take(6)

Out[ 19 ]: 
[('Arthur Conan Doyle', ['The Sign of Four']),
 ("Louis L'Amour", ['To Tame a Land', "The Warrior's Path"]),
 ('P.G. Wodehouse', ['Pigs Have Wings', 'The Return of Jeeves']),
 ('R.K. Narayan', ['Swami and Friends'])]

In [9]:
df1 = spark.read.csv("/FileStore/tables/authors.csv", header=True)
df2 = spark.read.csv("/FileStore/tables/books.csv", header=True)
df2.show()

+------+--------------------+
BookID| Title|
+------+--------------------+
 1|The Return of Jeeves|
 2| The Sign of Four|
 3| Pigs Have Wings|
 4| To Tame a Land|
 5| Swami and Friends|
 6| The Warrior's Path|
+------+--------------------+

In [10]:
from pyspark.sql.functions import collect_set
df1.join(df2, on = "BookID").select('Author','Title').groupBy('Author').agg(collect_set('Title').alias('Titles')).show(truncate=False)

+------------------+---------------------------------------+
Author |Titles |
+------------------+---------------------------------------+
Arthur Conan Doyle|[The Sign of Four] |
Louis L'Amour |[To Tame a Land, The Warrior's Path] |
P.G. Wodehouse |[The Return of Jeeves, Pigs Have Wings]|
R.K. Narayan |[Swami and Friends] |
+------------------+---------------------------------------+

In [11]:
#word count using dataframes
import pyspark.sql.functions as f

df_words = spark.read.text("/FileStore/tables/omar.txt")
df_words.withColumn('value',f.explode(f.split(f.col('value'), ' '))).withColumnRenamed('value','Word').groupBy('Word').count().sort('count',ascending=False).show()


+--------+-----+
 Word|count|
+--------+-----+
 test| 4|
 and| 3|
 more| 3|
 the| 2|
 One| 2|
 in| 2|
 is| 2|
 line| 2|
 exam| 1|
 for| 1|
Omarmart| 1|
 Omar| 1|
 Where| 1|
 no| 1|
 come| 1|
 another| 1|
 yet| 1|
 to| 1|
+--------+-----+